# RTW MAM Model V1 
# County Data First

In [91]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import csv
import pickle

In [187]:
#SET DATE FILTERS

prevalence = ['2020-06-17','2020-06-16','2020-06-15','2020-06-14','2020-06-13','2020-06-12','2020-06-11','2020-06-10','2020-06-09','2020-06-08',
              '2020-06-07','2020-06-06','2020-06-05','2020-06-04',]

day7_current = ['2020-06-17','2020-06-16','2020-06-15','2020-06-14','2020-06-13','2020-06-12','2020-06-11',]

day7_previous = ['2020-06-10','2020-06-09','2020-06-08','2020-06-07','2020-06-06','2020-06-05','2020-06-04']

curr_test_start = [int(20200617)]
curr_test_end = [int(20200611)]
prev_test_start = [int(20200610)]
prev_test_end = [int(20200604)]

In [107]:
#Creating pop file
#df = pd.read_csv("County4techs_4_15.csv",names=["I", "day", "cases", "county","state","doubling","fips","pop"],
#                dtype={'fips': str})

#df_filtered = df[(df.day==5)]
#print(df.shape)

#pop_df = df_filtered[['fips','county','state','pop' ]]
#print(pop_df.head())

#pickle.dump(pop_df, open("pop_df.sav","wb" ))

    fips       county       state         pop
1  53061    Snohomish  Washington    822083.0
3  17031         Cook    Illinois   5150233.0
5  06059       Orange  California   3175692.0
7  04013     Maricopa     Arizona   4485414.0
9  06037  Los Angeles  California  10039107.0


In [144]:
#UPDATES NEEDED - bring in base file, within base file add in state 2 digit codes


#Bring in County Data set
nytimes = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
counties = pd.read_csv(nytimes,dtype={'fips': str})
print(nyt.head())

#Change to own pop_df location
pop_df = pickle.load(open("pop_df.sav","rb"))

print(pop_df.head())
print(type(pop_df))

         date     county       state   fips  cases  deaths
0  2020-01-21  Snohomish  Washington  53061      1       0
1  2020-01-22  Snohomish  Washington  53061      1       0
2  2020-01-23  Snohomish  Washington  53061      1       0
3  2020-01-24       Cook    Illinois  17031      1       0
4  2020-01-24  Snohomish  Washington  53061      1       0
    fips       county       state         pop
1  53061    Snohomish  Washington    822083.0
3  17031         Cook    Illinois   5150233.0
5  06059       Orange  California   3175692.0
7  04013     Maricopa     Arizona   4485414.0
9  06037  Los Angeles  California  10039107.0
<class 'pandas.core.frame.DataFrame'>


# Create Active Case Count

In [145]:
#Create new cases field
#Sort df by Fips and Date
counties.sort_values(by=['fips','date'],inplace=True, ascending=True)
print(counties.head())

#Take difference between rows for new case and new death numbers
counties['case_diff'] = counties.cases.diff()
counties['death_diff'] = counties.deaths.diff()


print(counties.head())

             date   county    state   fips  cases  deaths
9480   2020-03-24  Autauga  Alabama  01001      1       0
10832  2020-03-25  Autauga  Alabama  01001      4       0
12361  2020-03-26  Autauga  Alabama  01001      6       0
14021  2020-03-27  Autauga  Alabama  01001      6       0
15801  2020-03-28  Autauga  Alabama  01001      6       0
             date   county    state   fips  cases  deaths  case_diff  \
9480   2020-03-24  Autauga  Alabama  01001      1       0        NaN   
10832  2020-03-25  Autauga  Alabama  01001      4       0        3.0   
12361  2020-03-26  Autauga  Alabama  01001      6       0        2.0   
14021  2020-03-27  Autauga  Alabama  01001      6       0        0.0   
15801  2020-03-28  Autauga  Alabama  01001      6       0        0.0   

       death_diff  
9480          NaN  
10832         0.0  
12361         0.0  
14021         0.0  
15801         0.0  


In [146]:
#Filter for past 14 days
range_14 = counties[counties['date'].isin(prevalence)]
print(range_14.head(15))

              date   county    state   fips  cases  deaths  case_diff  \
200589  2020-06-03  Autauga  Alabama  01001    240       5        0.0   
203588  2020-06-04  Autauga  Alabama  01001    242       5        2.0   
206590  2020-06-05  Autauga  Alabama  01001    249       5        7.0   
209596  2020-06-06  Autauga  Alabama  01001    260       5       11.0   
212603  2020-06-07  Autauga  Alabama  01001    266       5        6.0   
215613  2020-06-08  Autauga  Alabama  01001    273       5        7.0   
218625  2020-06-09  Autauga  Alabama  01001    283       5       10.0   
221641  2020-06-10  Autauga  Alabama  01001    296       6       13.0   
224663  2020-06-11  Autauga  Alabama  01001    316       6       20.0   
227686  2020-06-12  Autauga  Alabama  01001    327       6       11.0   
230711  2020-06-13  Autauga  Alabama  01001    335       6        8.0   
233737  2020-06-14  Autauga  Alabama  01001    361       6       26.0   
236764  2020-06-15  Autauga  Alabama  01001    368 

In [147]:
#Group by FIPS, sum up new Cases for prevalence - for 7 day averages will take mean after filtering for 7 days
prevalence_grouped = range_14.groupby(['fips'], as_index=False).sum()

print(prevalence_grouped.head(25))

     fips  cases  deaths  case_diff  death_diff
0   01001   4189      78      133.0         2.0
1   01003   4852     126       81.0         0.0
2   01005   2914      14       69.0         0.0
3   01007   1292      14       37.0         0.0
4   01009   1214      14       56.0         0.0
5   01011   3562     114       96.0         3.0
6   01013   6711     323      133.0         7.0
7   01015   2654      43       33.0         1.0
8   01017   5858     364      115.0         0.0
9   01019    631      58       14.0         2.0
10  01021   1736      25       36.0         1.0
11  01023   2276     147       24.0         1.0
12  01025   2342      37       46.0         1.0
13  01027    411      28        2.0         0.0
14  01029    260      14        7.0         0.0
15  01031   3672      14       59.0         0.0
16  01033   3225      58       86.0         3.0
17  01035    939      14       68.0         0.0
18  01037    590      14        6.0         0.0
19  01039   1592      14       99.0     

In [148]:
#Join in Prevalence to 
prevalence_w_pop = pd.merge(pop_df,
                 prevalence_grouped[['fips','case_diff']],
                 on='fips', 
                how='left')
print(prevalence_w_pop.head(10))

    fips         county          state         pop  case_diff
0  53061      Snohomish     Washington    822083.0      202.0
1  17031           Cook       Illinois   5150233.0     5865.0
2  06059         Orange     California   3175692.0     2359.0
3  04013       Maricopa        Arizona   4485414.0    10239.0
4  06037    Los Angeles     California  10039107.0    17966.0
5  06085    Santa Clara     California   1927852.0      435.0
6  25025        Suffolk  Massachusetts    803907.0      731.0
7  06075  San Francisco     California    881549.0      384.0
8  55025           Dane      Wisconsin    546695.0      226.0
9  06073      San Diego     California   3338330.0     2056.0


In [149]:
prevalence_w_pop['active_cases_100k'] = ((prevalence_w_pop['case_diff']/prevalence_w_pop['pop'])*100000)
print(prevalence_w_pop.head())

    fips       county       state         pop  case_diff  active_cases_100k
0  53061    Snohomish  Washington    822083.0      202.0          24.571728
1  17031         Cook    Illinois   5150233.0     5865.0         113.878343
2  06059       Orange  California   3175692.0     2359.0          74.283022
3  04013     Maricopa     Arizona   4485414.0    10239.0         228.273243
4  06037  Los Angeles  California  10039107.0    17966.0         178.960141


# Create 7 Day Rolling Avgs for Cases & Deaths

In [150]:
#Filter Diff datasets for current and previous 7 day periods

#join prev and current together with pop

range_current7 = counties[counties['date'].isin(day7_current)]
range_prev7 = counties[counties['date'].isin(day7_previous)]

print(range_current7.head())

              date   county    state   fips  cases  deaths  case_diff  \
221641  2020-06-10  Autauga  Alabama  01001    296       6       13.0   
224663  2020-06-11  Autauga  Alabama  01001    316       6       20.0   
227686  2020-06-12  Autauga  Alabama  01001    327       6       11.0   
230711  2020-06-13  Autauga  Alabama  01001    335       6        8.0   
233737  2020-06-14  Autauga  Alabama  01001    361       6       26.0   

        death_diff  
221641         1.0  
224663         0.0  
227686         0.0  
230711         0.0  
233737         0.0  


In [151]:
#Group by with mean
current_grouped = range_current7.groupby(['fips'],as_index=False).mean()
previous_grouped = range_prev7.groupby(['fips'], as_index=False).mean()

current_grouped['curr7_case'] = current_grouped['case_diff']
current_grouped['curr7_death'] = current_grouped['death_diff']
previous_grouped['prev7_case'] = previous_grouped['case_diff']
previous_grouped['prev7_death'] = previous_grouped['death_diff']
print(current_grouped.head())
print(previous_grouped.head())

    fips       cases    deaths  case_diff  death_diff  curr7_case  curr7_death
0  01001  339.428571  6.142857  12.857143    0.285714   12.857143     0.285714
1  01003  371.285714  9.000000   7.000000    0.000000    7.000000     0.000000
2  01005  227.285714  1.000000   6.428571    0.000000    6.428571     0.000000
3  01007  103.571429  1.000000   4.000000    0.000000    4.000000     0.000000
4  01009  102.857143  1.000000   6.285714    0.000000    6.285714     0.000000
    fips       cases  deaths  case_diff  death_diff  prev7_case  prev7_death
0  01001  259.000000     5.0   6.142857         0.0    6.142857          0.0
1  01003  321.857143     9.0   4.571429         0.0    4.571429          0.0
2  01005  189.000000     1.0   3.428571         0.0    3.428571          0.0
3  01007   81.000000     1.0   1.285714         0.0    1.285714          0.0
4  01009   70.571429     1.0   1.714286         0.0    1.714286          0.0


In [152]:
#Merge both sets with population
current_w_pop = pd.merge(pop_df,
                 current_grouped[['fips','curr7_case', 'curr7_death']],
                 on='fips', 
                how='left')
print(current_w_pop.head(10))

previous_w_pop = pd.merge(pop_df,
                 previous_grouped[['fips','prev7_case', 'prev7_death']],
                 on='fips', 
                how='left')
print(previous_w_pop.head(10))

    fips         county          state         pop   curr7_case  curr7_death
0  53061      Snohomish     Washington    822083.0    12.857143     0.428571
1  17031           Cook       Illinois   5150233.0   323.857143    34.714286
2  06059         Orange     California   3175692.0   173.285714     6.857143
3  04013       Maricopa        Arizona   4485414.0   914.428571     9.142857
4  06037    Los Angeles     California  10039107.0  1323.142857    36.000000
5  06085    Santa Clara     California   1927852.0    33.857143     0.428571
6  25025        Suffolk  Massachusetts    803907.0    42.857143     2.428571
7  06075  San Francisco     California    881549.0    25.428571     0.285714
8  55025           Dane      Wisconsin    546695.0    16.285714     0.000000
9  06073      San Diego     California   3338330.0   143.000000     1.571429
    fips         county          state         pop   prev7_case  prev7_death
0  53061      Snohomish     Washington    822083.0    16.000000     0.714286

In [153]:
current_w_pop['cur_7rollavg_cases'] = ((current_w_pop['curr7_case']/current_w_pop['pop'])*100000)
current_w_pop['cur_7rollavg_deaths'] = ((current_w_pop['curr7_death']/current_w_pop['pop'])*100000)

In [154]:
previous_w_pop['prev_7rollavg_cases'] = ((previous_w_pop['prev7_case']/previous_w_pop['pop'])*100000)
previous_w_pop['prev_7rollavg_deaths'] = ((previous_w_pop['prev7_death']/previous_w_pop['pop'])*100000)

In [155]:
#Merge with prevalence dataset
prev_curr = pd.merge(prevalence_w_pop,
                 current_w_pop[['fips','cur_7rollavg_cases', 'cur_7rollavg_deaths','curr7_case','curr7_death']],
                 on='fips', 
                how='left')
print(prev_curr.head(10))

    fips         county          state         pop  case_diff  \
0  53061      Snohomish     Washington    822083.0      202.0   
1  17031           Cook       Illinois   5150233.0     5865.0   
2  06059         Orange     California   3175692.0     2359.0   
3  04013       Maricopa        Arizona   4485414.0    10239.0   
4  06037    Los Angeles     California  10039107.0    17966.0   
5  06085    Santa Clara     California   1927852.0      435.0   
6  25025        Suffolk  Massachusetts    803907.0      731.0   
7  06075  San Francisco     California    881549.0      384.0   
8  55025           Dane      Wisconsin    546695.0      226.0   
9  06073      San Diego     California   3338330.0     2056.0   

   active_cases_100k  cur_7rollavg_cases  cur_7rollavg_deaths   curr7_case  \
0          24.571728            1.563971             0.052132    12.857143   
1         113.878343            6.288204             0.674033   323.857143   
2          74.283022            5.456628          

In [156]:
county_set = pd.merge(prev_curr,
                 previous_w_pop[['fips','prev_7rollavg_cases', 'prev_7rollavg_deaths','prev7_case','prev7_death']],
                 on='fips', 
                how='left')
print(county_set.head(10))

    fips         county          state         pop  case_diff  \
0  53061      Snohomish     Washington    822083.0      202.0   
1  17031           Cook       Illinois   5150233.0     5865.0   
2  06059         Orange     California   3175692.0     2359.0   
3  04013       Maricopa        Arizona   4485414.0    10239.0   
4  06037    Los Angeles     California  10039107.0    17966.0   
5  06085    Santa Clara     California   1927852.0      435.0   
6  25025        Suffolk  Massachusetts    803907.0      731.0   
7  06075  San Francisco     California    881549.0      384.0   
8  55025           Dane      Wisconsin    546695.0      226.0   
9  06073      San Diego     California   3338330.0     2056.0   

   active_cases_100k  cur_7rollavg_cases  cur_7rollavg_deaths   curr7_case  \
0          24.571728            1.563971             0.052132    12.857143   
1         113.878343            6.288204             0.674033   323.857143   
2          74.283022            5.456628          

In [ ]:
#Create indicators for rolling averages
county_set['roll7case_diff'] = ((county_set['cur_7rollavg_cases']-county_set['prev_7rollavg_cases'])/county_set['prev_7rollavg_cases'])
county_set['roll7death_diff'] = ((county_set['cur_7rollavg_deaths']-county_set['prev_7rollavg_deaths'])/county_set['prev_7rollavg_deaths'])

print(county_set.head())

In [158]:
#Write to CSV
#county_set.to_csv('CountyMAMV2_618.csv') 

# State Level Data

In [159]:
# Pull in NYTimes state data, diff, do same 7 day rolling avg counts
# pull in COVID tracking test data. calculate pos % then rolling avgs
# Manually pull in Rt, might drop this metric

In [162]:
#Bring in state level NYT Data
nytimes_state = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv"
state= pd.read_csv(nytimes_state)
print(nyt_state.head())

         date       state  fips  cases  deaths
0  2020-01-21  Washington    53      1       0
1  2020-01-22  Washington    53      1       0
2  2020-01-23  Washington    53      1       0
3  2020-01-24    Illinois    17      1       0
4  2020-01-24  Washington    53      1       0


In [164]:
#Create new cases field
#Sort df by Fips and Date
state.sort_values(by=['fips','date'],inplace=True, ascending=True)
print(state.head())

#Take difference between rows for new case and new death numbers
state['case_diff'] = state.cases.diff()
print(state.head())

           date    state  fips  cases  deaths  case_diff
586  2020-03-13  Alabama     1      6       0        NaN
637  2020-03-14  Alabama     1     12       0        6.0
689  2020-03-15  Alabama     1     23       0       11.0
742  2020-03-16  Alabama     1     29       0        6.0
795  2020-03-17  Alabama     1     39       0       10.0
           date    state  fips  cases  deaths  case_diff
586  2020-03-13  Alabama     1      6       0        NaN
637  2020-03-14  Alabama     1     12       0        6.0
689  2020-03-15  Alabama     1     23       0       11.0
742  2020-03-16  Alabama     1     29       0        6.0
795  2020-03-17  Alabama     1     39       0       10.0


In [166]:
#Take Current and previous rolling averages
state_range_current7 = state[state['date'].isin(day7_current)]
state_range_prev7 = state[state['date'].isin(day7_previous)]

print(state_range_current7.head(8))

            date    state  fips  cases  deaths  case_diff
5459  2020-06-10  Alabama     1  21989     744      567.0
5514  2020-06-11  Alabama     1  22845     755      856.0
5569  2020-06-12  Alabama     1  23710     769      865.0
5624  2020-06-13  Alabama     1  24601     773      891.0
5679  2020-06-14  Alabama     1  25615     773     1014.0
5734  2020-06-15  Alabama     1  26272     774      657.0
5789  2020-06-16  Alabama     1  26912     785      640.0
5460  2020-06-10   Alaska     2    642       9       22.0


In [167]:
###PLACE HOLDER FOR MERGING TO BASE FILE####

In [176]:
#########################################
#Bring in COVID TRACKING data
url_all = "https://covidtracking.com/api/v1/states/daily.csv"
testing_all = pd.read_csv(url_all)
testing = testing_all[['date','state','positiveIncrease','negativeIncrease', 'totalTestResultsIncrease']]

print(testing.head())
print(testing.dtypes)

       date state  positiveIncrease  negativeIncrease  \
0  20200618    AK                12              3464   
1  20200618    AL               894             10917   
2  20200618    AR               322              7413   
3  20200618    AS                 0                 0   
4  20200618    AZ              2519             11020   

   totalTestResultsIncrease  
0                      3476  
1                     11811  
2                      7735  
3                         0  
4                     13539  
date                         int64
state                       object
positiveIncrease             int64
negativeIncrease             int64
totalTestResultsIncrease     int64
dtype: object


In [188]:
curr_testing = testing[(testing['date']>=curr_test_end) & (testing['date']<=curr_test_start)]
print(curr_testing.head(10))

prev_testing = testing[(testing['date']>=prev_test_end) & (testing['date']<=prev_test_start)]
print(prev_testing.head())

        date state  positiveIncrease  negativeIncrease  \
56  20200617    AK                20              1474   
57  20200617    AL               400              4899   
58  20200617    AR               415             14718   
59  20200617    AS                 0                 0   
60  20200617    AZ              1827             10032   
61  20200617    CA              3455             56778   
62  20200617    CO               143              3368   
63  20200617    CT                80              6358   
64  20200617    DC                29              2931   
65  20200617    DE                41               472   

    totalTestResultsIncrease  
56                      1494  
57                      5299  
58                     15133  
59                         0  
60                     11859  
61                     60233  
62                      3511  
63                      6438  
64                      2960  
65                       513  
         date state 